In [21]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Finding the frequently asked questions and answers on github
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
# Request these document to fetch them into the code
docs_response = requests.get(docs_url)
# Create the json of the request response
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Examine the final json
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
# Create a pandas dataframe based on this JSON
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
# See the first 5 rows of the dataframe
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [4]:
# Limit the questions and answers to a specific course
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [7]:
# Vectorize the documents that we want to search

In [5]:
# Use Bag of words to vectorize the text
from sklearn.feature_extraction.text import CountVectorizer

# Initializw the vectorizer model
cv = CountVectorizer(stop_words='english', min_df = 5)
# Fit the vectorizer with the data to convert them to vectors
X = cv.fit_transform(df['text'])
# Take the words appearing in the text as column names
names = cv.get_feature_names_out()

# Create the dataframe from this text
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0,0,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
yml,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
youtube,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Vectorizing the text using TD-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(stop_words='english', min_df = 5)
X = cv.fit_transform(df['text'])

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4,5,6,7,8,9,...,938,939,940,941,942,943,944,945,946,947
01,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
02,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
03,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
04,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
05,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0.0,0.00,0.28,0.0,0.0,0.00,0.21,0.2,0.15,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
yml,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.11,0.0,0.0,0.0,0.00
youtube,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.15,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
zip,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00


# Try to find an answer, searching only in the answer body

For that we will try to find query to document similarities by:
- Vectorizing the query using the vectorizer / dictionary of words that we fit with our documents
- Find the dot product of the vector/query and all the documents to find a similarity score of each document with the query -> cosine similarity
- Then we short the documents based on their scores and retrieve the more relevant ones to our query

In [15]:
# User query
query = "Do I need to know python to sign up for the January course?"

# Vectorize the query using the fitted vectorizer
q = cv.transform([query]) # need to be a list
# See the array of the query's vector reprensentation
print(q.toarray())
# See the words that has weight in the query's vector
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

[[0. 0. 0. ... 0. 0. 0.]]


{'01': 0.0,
 '02': 0.0,
 '03': 0.0,
 '04': 0.0,
 '05': 0.0,
 '06': 0.0,
 '09': 0.0,
 '10': 0.0,
 '100': 0.0,
 '11': 0.0,
 '12': 0.0,
 '127': 0.0,
 '13': 0.0,
 '14': 0.0,
 '15': 0.0,
 '16': 0.0,
 '17': 0.0,
 '19': 0.0,
 '1st': 0.0,
 '20': 0.0,
 '2019': 0.0,
 '2020': 0.0,
 '2021': 0.0,
 '2022': 0.0,
 '2023': 0.0,
 '2024': 0.0,
 '21': 0.0,
 '22': 0.0,
 '24': 0.0,
 '25': 0.0,
 '2pacx': 0.0,
 '30': 0.0,
 '35': 0.0,
 '403': 0.0,
 '42': 0.0,
 '50': 0.0,
 '5000': 0.0,
 '5431': 0.0,
 '5432': 0.0,
 '60': 0.0,
 '600': 0.0,
 '7077': 0.0,
 '80': 0.0,
 '8080': 0.0,
 '8888': 0.0,
 '9696': 0.0,
 'abhijit': 0.0,
 'able': 0.0,
 'abolade': 0.0,
 'absolute': 0.0,
 'accept': 0.0,
 'access': 0.0,
 'accordingly': 0.0,
 'account': 0.0,
 'accuracy': 0.0,
 'action': 0.0,
 'activate': 0.0,
 'actual': 0.0,
 'actually': 0.0,
 'add': 0.0,
 'added': 0.0,
 'adding': 0.0,
 'addition': 0.0,
 'additional': 0.0,
 'additionally': 0.0,
 'address': 0.0,
 'admin': 0.0,
 'advani': 0.0,
 'ahmed': 0.0,
 'ai': 0.0,
 'airflow': 0

In [36]:
# Calculate the similarity between the query and all documents using the dot product
similarity_scores = X.dot(q.T).toarray() # need to transpose the query before using the dot product
# Then we can sort these similarities and find the indexes of the most relevant documents
indxs = np.argsort(similarity_scores.flatten())[-5:]
print(indxs)

[764  27 806 577 445]


In [38]:
# See the most relevant answer
df['text'].iloc[764]

'If you have submitted two projects (and peer-reviewed at least 3 course-mates’ projects for each submission), you will get the certificate for the course. According to the course coordinator, Alexey Grigorev, only two projects are needed to get the course certificate.\n(optional) David Odimegwu'

### We can calculate the scores using directly the cosine_similarity from Sklearn

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the scores using cosine similarity
similarity_scores = cosine_similarity(X,q).flatten()
# Sort them by similarity score and find the indexes 
indxs = np.argsort(similarity_scores)[-5:]
print(indxs)

[764  27 806 577 445]


### Find the similarities scores for all the fields in the questions

In [43]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

# For each field in the dataframe convert it into vectors
for field in fields:
    # Fit the vectorizer 
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    # Vectorize the text
    X = cv.fit_transform(df[field])

    # Created a dictionary of all the fitted vectorizers
    transformers[field] = cv
    # Create a dictionary with all the matrices of vectors
    matrices[field] = X

transformers['text'].get_feature_names_out()
matrices['text']

<948x2118 sparse matrix of type '<class 'numpy.float64'>'
	with 26463 stored elements in Compressed Sparse Row format>

In [48]:
# Vectorize a query and search for all the fields

# Initialize the array of scores with zeros
score = np.zeros(len(df))

query = 'I just discovered the course, is it too late to join?'

# Search for all the fields and aggregate the scores
for f in fields:
    # Create the query vector using the previous transformers
    q = transformers[f].transform([query])
    # Fetch the matrices
    X = matrices[f]
    # Calculate the cosine similarity for the field
    f_score = cosine_similarity(X,q).flatten()
    # Append this into the score
    score = score + f_score

# Find the indexes of the most relevant documents
indxs = np.argsort(score)[-5:]
# display the relevant answers
df.iloc[indxs]

,course,section,question,text
5,data-engineering-zoomcamp,General course-related questions,Course - how many Zoomcamps in a year?,"There are 3 Zoom Camps in a year, as of 2024. ..."
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
9,data-engineering-zoomcamp,General course-related questions,Course - Which playlist on YouTube should I re...,All the main videos are stored in the Main “DA...
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...


948